In [ ]:
!pip install rasterio
import rasterio
from rasterio.windows import Window
import os
from google.colab import drive
import sys
import numpy as np
import rasterio
from rasterio.windows import Window
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 67.6 MB/s eta 0:00:00


In [ ]:
#drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#path = '/content/drive/MyDrive/Academico/Computer Vision'
#sys.path.append(path)
base_dir = os.getcwd()

In [ ]:
def create_json_metadata(image_dir, json_path):
    data = []
    for root, dirs, files in os.walk(image_dir):
        for file in files:
            if file.endswith('.tif'):
                label = 1 if file.startswith('vya_') else 0
                data.append({
                    'image_path': os.path.join(root, file),
                    'label': label
                })
    with open(json_path, 'w') as f:
        json.dump(data, f)

# Paths to your image directories and output JSON files
train_image_directory = base_dir + '/Data/Train_80'
test_image_directory = base_dir + '/Data/Test_20'

train_json_output_path = base_dir + '/Data/Train_80.json'
test_json_output_path = base_dir + '/Data/Test_20.json'

# Create JSON metadata files for training and testing datasets
create_json_metadata(train_image_directory, train_json_output_path)
create_json_metadata(test_image_directory, test_json_output_path)

In [ ]:



with rasterio.open(base_dir + '/Data/Sentinel2_Buenos_Aires.tif') as src:


    width = src.width
    height = src.height


    tile_size = 32

    for i in range(0, width, tile_size):
        for j in range(0, height, tile_size):
          if i + tile_size <= width and j + tile_size <= height:
            window = Window(i, j, tile_size, tile_size)
            transform = src.window_transform(window)
            tile = src.read(window=window)

            # Guardar cada recorte
            tile_filename = f'{base_dir}/Data/Sentinel2/tile_{i}_{j}.tif'
            with rasterio.open(
                tile_filename, 'w',
                driver='GTiff',
                height=tile.shape[1],
                width=tile.shape[2],
                count=src.count,
                dtype=tile.dtype,
                crs=src.crs,
                transform=transform,
            ) as dst:
                dst.write(tile)

In [ ]:



output_dir = '/Data/Sentinel2'


num_files = len([name for name in os.listdir(output_dir) if os.path.isfile(os.path.join(output_dir, name))])

print(f'Número de imágenes en la carpeta: {num_files}')


Número de imágenes en la carpeta: 3534


In [ ]:
import os
import torch
from PIL import Image
import numpy as np
from RealESRGAN import RealESRGAN

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = RealESRGAN(device, scale=4)
model.load_weights(base_dir + '/Data/weights/RealESRGAN_x4.pth')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:671: FutureWarning: 'cached_download' is the legacy way to download files from the HF hub, please consider upgrading to 'hf_hub_download'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


RealESRGAN_x4.pth:   0%|          | 0.00/67.0M [00:00<?, ?B/s]

Weights downloaded to: weights/RealESRGAN_x4.pth


In [ ]:
import rasterio
import matplotlib.pyplot as plt
def enhance_image(image_path, model):

    with rasterio.open(image_path) as src:
        img = src.read([1, 2, 3])
        img = np.transpose(img, (1, 2, 0))


    if img.dtype != np.uint8:
        img = ((img-img.min())* (255 / (img.max() - img.min()))).astype(np.uint8)

    img = Image.fromarray(img)


    sr_image = model.predict(img)


    return sr_image


input_dir = base_dir + '/Data/Train_80'
output_dir = base_dir +  '/Data/Train_80_enhanced'

for filename in os.listdir(input_dir):
    if filename.endswith(".tif"):
        input_path = os.path.join(input_dir, filename)
        output_path = os.path.join(output_dir, filename.replace('.tif', '_enhanced.png'))


        enhanced_image = enhance_image(input_path, model)


        enhanced_image.save(output_path)

        print(f"Processed {filename} and saved enhanced image to {output_path}")

print("All images have been processed and saved.")

Processed vya_T21HUB_2560_2592_2176_2208.tif and saved enhanced image to /content/drive/MyDrive/Academico/Computer Vision/Train_80_quarter_enhanced/vya_T21HUB_2560_2592_2176_2208_enhanced.png
Processed T21HUB_672_704_4896_4928.tif and saved enhanced image to /content/drive/MyDrive/Academico/Computer Vision/Train_80_quarter_enhanced/T21HUB_672_704_4896_4928_enhanced.png
Processed T21HUB_6656_6688_9696_9728.tif and saved enhanced image to /content/drive/MyDrive/Academico/Computer Vision/Train_80_quarter_enhanced/T21HUB_6656_6688_9696_9728_enhanced.png
Processed vya_T21HUB_3008_3040_3392_3424.tif and saved enhanced image to /content/drive/MyDrive/Academico/Computer Vision/Train_80_quarter_enhanced/vya_T21HUB_3008_3040_3392_3424_enhanced.png
Processed T21HUB_6080_6112_4416_4448.tif and saved enhanced image to /content/drive/MyDrive/Academico/Computer Vision/Train_80_quarter_enhanced/T21HUB_6080_6112_4416_4448_enhanced.png
Processed vya_T21HUB_5760_5792_5056_5088.tif and saved enhanced imag

In [ ]:

import rasterio
import matplotlib.pyplot as plt
def enhance_image(image_path, model):

    with rasterio.open(image_path) as src:
        img = src.read([1, 2, 3])
        img = np.transpose(img, (1, 2, 0))


    if img.dtype != np.uint8:
        img = ((img-img.min())* (255 / (img.max() - img.min()))).astype(np.uint8)

    img = Image.fromarray(img)
    sr_image = model.predict(img)


    return sr_image


input_dir = base_dir + '/Data/Test_20'
output_dir = base_dir + '/Data/Test_20_enhanced'
# Enhance the quality of the image
for filename in os.listdir(input_dir):
    if filename.endswith(".tif"):  # Filtrar solo archivos TIFF
        input_path = os.path.join(input_dir, filename)
        output_path = os.path.join(output_dir, filename.replace('.tif', '_enhanced.png'))  # Guardar como PNG

        # Mejorar la calidad de la imagen
        enhanced_image = enhance_image(input_path, model)

        # Guardar la imagen mejorada
        enhanced_image.save(output_path)

        print(f"Processed {filename} and saved enhanced image to {output_path}")

print("All images have been processed and saved.")

Processed T21HUB_2048_2080_4832_4864.tif and saved enhanced image to /content/drive/MyDrive/Academico/Computer Vision/Test_20_quarter_enhanced/T21HUB_2048_2080_4832_4864_enhanced.png
Processed vya_T21HUB_6560_6592_5184_5216.tif and saved enhanced image to /content/drive/MyDrive/Academico/Computer Vision/Test_20_quarter_enhanced/vya_T21HUB_6560_6592_5184_5216_enhanced.png
Processed T21HUB_2048_2080_5280_5312.tif and saved enhanced image to /content/drive/MyDrive/Academico/Computer Vision/Test_20_quarter_enhanced/T21HUB_2048_2080_5280_5312_enhanced.png
Processed T21HUB_800_832_4864_4896.tif and saved enhanced image to /content/drive/MyDrive/Academico/Computer Vision/Test_20_quarter_enhanced/T21HUB_800_832_4864_4896_enhanced.png
Processed T21HUB_2944_2976_2848_2880.tif and saved enhanced image to /content/drive/MyDrive/Academico/Computer Vision/Test_20_quarter_enhanced/T21HUB_2944_2976_2848_2880_enhanced.png
Processed vya_T21HUB_2528_2560_3488_3520.tif and saved enhanced image to /content

In [ ]:
import json
def create_json_metadata(image_dir, json_path):
    data = []
    for root, dirs, files in os.walk(image_dir):

        for file in files:
            if file.endswith('.png'):

                label = 1 if file.startswith('vya_') else 0
                data.append({
                    'image_path': os.path.join(root, file),
                    'label': label
                })
    with open(json_path, 'w') as f:
        json.dump(data, f)

# Paths to your image directories and output JSON files
train_image_directory = base_dir + '/Data/Train_80_enhanced'
test_image_directory = base_dir + '/Data/Test_20_enhanced'

train_json_output_path = base_dir + '/Data/Train_80_enhanced.json'
test_json_output_path = base_dir + '/Data/Test_20_enhanced.json'

# Create JSON metadata files for training and testing datasets
create_json_metadata(train_image_directory, train_json_output_path)
create_json_metadata(test_image_directory, test_json_output_path)

In [ ]:

import rasterio
import matplotlib.pyplot as plt
def enhance_image(image_path, model):
    # Read and preprocess the image using rasterio
    with rasterio.open(image_path) as src:
        img = src.read([1, 2, 3])  # Read the first three bands (assuming RGB)
        img = np.transpose(img, (1, 2, 0))  # Convert to HWC format

    # Convert to 8-bit image if necessary
    if img.dtype != np.uint8:
        img = ((img-img.min())* (255 / (img.max() - img.min()))).astype(np.uint8)

    img = Image.fromarray(img)

    # Predict the super-resolved image
    sr_image = model.predict(img)


    return sr_image


input_dir = base_dir + '/Data/Sentinel2'
output_dir = base_dir + '/Data/Sentinel2_enhanced'
# Enhance the quality of the image
for filename in os.listdir(input_dir):
    if filename.endswith(".tif"):  # Filtrar solo archivos TIFF
        input_path = os.path.join(input_dir, filename)
        output_path = os.path.join(output_dir, filename.replace('.tif', '_enhanced.png'))  # Guardar como PNG

        # Mejorar la calidad de la imagen
        enhanced_image = enhance_image(input_path, model)

        # Guardar la imagen mejorada
        enhanced_image.save(output_path)

        print(f"Processed {filename} and saved enhanced image to {output_path}")

print("All images have been processed and saved.")

Processed tile_1280_1728.tif and saved enhanced image to /content/drive/MyDrive/Academico/Computer Vision/Sentinel2_enhanced/tile_1280_1728_enhanced.png
Processed tile_1280_1760.tif and saved enhanced image to /content/drive/MyDrive/Academico/Computer Vision/Sentinel2_enhanced/tile_1280_1760_enhanced.png
Processed tile_1280_1792.tif and saved enhanced image to /content/drive/MyDrive/Academico/Computer Vision/Sentinel2_enhanced/tile_1280_1792_enhanced.png
Processed tile_1280_1824.tif and saved enhanced image to /content/drive/MyDrive/Academico/Computer Vision/Sentinel2_enhanced/tile_1280_1824_enhanced.png
Processed tile_1280_1856.tif and saved enhanced image to /content/drive/MyDrive/Academico/Computer Vision/Sentinel2_enhanced/tile_1280_1856_enhanced.png
Processed tile_1280_1888.tif and saved enhanced image to /content/drive/MyDrive/Academico/Computer Vision/Sentinel2_enhanced/tile_1280_1888_enhanced.png
Processed tile_1280_1920.tif and saved enhanced image to /content/drive/MyDrive/Ac